In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [5]:
import glob
from PIL import Image
import numpy as np
import os
from tqdm import tqdm

from keras.models import Sequential, Model, load_model
from keras.layers import UpSampling2D, Conv2D, Activation, BatchNormalization, Reshape, Dense, Input, LeakyReLU, Dropout, Flatten, ZeroPadding2D
from keras.optimizers import Adam

In [3]:


class DCGAN:
    def __init__(self):
        self.discriminator_path = '/content/drive/My Drive/Colab Notebooks/model/discriminator.h5'
        self.generator_path = '/content/drive/My Drive/Colab Notebooks/model/generator.h5'
        self.output_directory = '/content/drive/My Drive/Colab Notebooks/gantest'
    
    def build_generator(self):
        noise_shape = (100,)
        model = Sequential()
        model.add(Dense(64 * (192 // (2 ** 5)) * (256 // (2 ** 5)), activation="relu", input_shape=noise_shape))
        model.add(Reshape(((192 // (2 ** 5)), (256 // (2 ** 5)), 64)))
        model.add(BatchNormalization(momentum=0.8))

        model.add(UpSampling2D())
        model.add(Conv2D(1024, (3, 3), padding = 'same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(momentum=0.8))

        model.add(UpSampling2D())
        model.add(Conv2D(512, (3, 3), padding = 'same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(momentum=0.8))

        model.add(UpSampling2D())
        model.add(Conv2D(256, (3, 3), padding = 'same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(momentum=0.8))

        model.add(UpSampling2D())
        model.add(Conv2D(128, (3, 3), padding = 'same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(momentum=0.8))

        model.add(UpSampling2D())
        model.add(Conv2D(64, (3, 3), padding = 'same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(32, (3, 3), padding = 'same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(3, (3, 3), padding = 'same'))
        model.add(Activation('relu'))

        noise = Input(shape = noise_shape)
        img = model(noise)

        return Model(noise, img)
    
    def build_discriminator(self):
        
        img_shape = (192, 256, 3)

        model = Sequential()

        model.add(Conv2D(32, (3, 3), strides = 2, input_shape=img_shape, padding = 'same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, (3, 3), strides = 2, padding = 'same'))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(128, (3, 3), strides = 2, padding = 'same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(256, (3, 3), strides = 1, padding = 'same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, (3, 3), strides = 1, padding = 'same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1, activation = 'sigmoid'))

        img = Input(shape = img_shape)
        result = model(img)

        return Model(img, result)
    
    def build_model(self):
        optimizer = Adam(0.0002, 0.5)
        if os.path.exists(self.discriminator_path) and os.path.exists(self.generator_path):
            self.discriminator = load_model(self.discriminator_path)
            self.generator = load_model(self.generator_path)
            print("Loaded models...")
        else:
            self.discriminator = self.build_discriminator()
            self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

            self.generator = self.build_generator()
            self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

            random_noise = Input(shape = (100,))
            img = self.generator(random_noise)

            self.discriminator.trainable = False

            result = self.discriminator(img)

            self.combined = Model(random_noise, result)
            self.combined.compile(loss = 'binary_crossentropy', optimizer=optimizer)

    def get_data(self,imgPath):
        basePath = '/content/drive/My Drive/Colab Notebooks/DCGAN-data/'
        X = []
        for path in tqdm(imgPath):
            img = Image.open(basePath + path)
            img = np.asarray(img)
            X.append(img)
        return np.asarray(X)
    
    def trainModel(self):
        self.build_model()
        pathList = os.listdir('/content/drive/My Drive/Colab Notebooks/DCGAN-data/')
        X_train = self.get_data(pathList)

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5

        batch_size = 32
        half_batch = batch_size // 2

        epochs = 500
        for epoch in range(epochs):

            noise = np.random.normal(0, 1, (batch_size, 100))
            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))

            index = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))
            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            print(f"{epoch} [D loss: {d_loss[0]} | D Accuracy: {d_loss[1]}] [G loss: {g_loss}]")

            if epoch % (50) == 0:
                self.save_imgs(epoch)
                self.discriminator.save(self.discriminator_path)
                self.generator.save(self.generator_path)
    def gene_imgs(self, count):
        noise = np.random.normal(0, 1, (count, 100))
        return self.generator.predict(noise)
    
    def save_imgs(self,epoch):
        r, c = 5, 5

        imgs = self.gene_imgs(r*c)
        imgs = 0.5 * imgs + 0.5

        nindex, height, width, intensity = imgs.shape
        nrows = nindex // c
        assert nindex == nrows * c
        
        gallery = (imgs.reshape(nrows, c, height, width, intensity)
                  .swapaxes(1, 2)
                  .reshape(height * nrows, width * c, intensity))
        path = self.output_directory + '/gallery'
        if not os.path.exists(path):
            os.makedirs(path)
        imsave(path + f"/{epoch}.jpg", gallery)

    def predict_img(self, count, threshold):
        self.build_model()

        imgs = []
        for i in range(count):
            score = [0]
            while not(threshold[0] < score[0] < threshold[1]):
                img = self.gene_imgs(1)
                score = self.discriminator.predict(img)
            print("Image found: ", score[0])
            imgs.append(img)

        imgs = np.asarray(imgs).squeeze()
        imgs = 0.5 * imgs + 0.5
        for i, img_array in enumerate(imgs):
            path = self.output_directory + '/predictimg'
            if not os.path.exists(path):
                os.makedirs(path)
            imsave(path + f"/{i}.png", img_array)


In [ ]:
dcgan = DCGAN()
dcgan.trainModel()